# BIBLIOTECA

In [3]:
pip install certifi

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip-system-certs


Usage:   
  /opt/anaconda3/bin/python -m pip <command> [options]

no such option: -s
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import ssl
import requests
import certifi
from io import StringIO
import datetime
import time

# WEB SCRAPING BCB ORIGINAL

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

: 

# SELIC - RESULTADOS DOS LEILÕES

In [ ]:
import ssl
import pandas as pd
from datetime import datetime

ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# 1. Reorganizar as colunas
colunas_desejadas = ['edital', 'tipoOferta', 'codigoTitulo', 'ofertante', 'prazo', 'dataLiquidacao','quantidadeOfertada', 'quantidadeAceita', 'taxaCorte']
tabela = tabela[colunas_desejadas]

# 2. Criar a coluna "NomeTitulo"
tabela['NomeTitulo'] = tabela['codigoTitulo'].map({210100: 'LFT 210100', 760199: 'NTN-B 760199'})

# 3. Mover a coluna "NomeTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('codigoTitulo') + 1, colunas.pop(colunas.index('NomeTitulo')))
tabela = tabela[colunas]

# 4. Ordenar por 'edital' com ordem personalizada
ordem_edital = [37, 38, 39]
tabela['edital'] = pd.Categorical(tabela['edital'], categories=ordem_edital, ordered=True)

# 5. Ordenar por 'prazo' com ordem personalizada
ordem_prazo = sorted(tabela['prazo'].unique())
tabela['prazo'] = pd.Categorical(tabela['prazo'], categories=ordem_prazo, ordered=True)

# 6. Criar a coluna "AnoTitulo"
tabela['AnoTitulo'] = (datetime.now() + pd.to_timedelta(pd.to_numeric(tabela['prazo'], errors='coerce'), unit='D')).dt.year

# 7. Mover a coluna "AnoTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('NomeTitulo') + 1, colunas.pop(colunas.index('AnoTitulo')))
tabela = tabela[colunas]

# 8. Formatar a coluna "taxaCorte" como porcentagem
tabela['taxaCorte'] = pd.to_numeric(tabela['taxaCorte'], errors='coerce') / 10000
tabela['taxaCorte'] = tabela['taxaCorte'].astype(str) + '%'

# 9. Formatar a coluna "quantidadeAceita"
tabela['quantidadeAceita'] = tabela['quantidadeAceita'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))


# 10. Formatar a coluna "quantidadeOfertada"
tabela['quantidadeOfertada'] = tabela['quantidadeOfertada'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))


# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

,edital,tipoOferta,codigoTitulo,NomeTitulo,AnoTitulo,ofertante,prazo,dataLiquidacao,quantidadeOfertada,quantidadeAceita,taxaCorte
0,NaN,Venda,100000,NaN,2026,Tesouro Nacional,383,2025-03-14 00:00:00,6.000.000,6.000.000,14.7499%
1,NaN,Venda,950199,NaN,2035,Tesouro Nacional,3580,2025-03-14 00:00:00,3.000.000,3.000.000,15.0275%
2,NaN,Venda,950199,NaN,2031,Tesouro Nacional,2119,2025-03-14 00:00:00,3.000.000,3.000.000,1.4995%
3,NaN,Venda,100000,NaN,2032,Tesouro Nacional,2484,2025-03-14 00:00:00,3.000.000,3.000.000,14.9424%
4,NaN,Venda,100000,NaN,2029,Tesouro Nacional,1389,2025-03-14 00:00:00,12.000.000,12.000.000,14.7097%


### Gráfico da Taxa de Corte

In [6]:
import plotly.graph_objects as go

# Garantir que a coluna 'taxaCorte' seja convertida corretamente
if tabela['taxaCorte'].dtype == 'object':  # Verifica se a coluna é do tipo string
    tabela['taxaCorte'] = tabela['taxaCorte'].str.rstrip('%').astype(float)

# Garantir que a coluna de datas ('dataLiquidacao') esteja no formato datetime
tabela['dataLiquidacao'] = pd.to_datetime(tabela['dataLiquidacao'], errors='coerce')

# Selecionar o maior valor de 'taxaCorte' para cada data
picos_por_data = tabela.groupby('dataLiquidacao', as_index=False).max()

# Criar o gráfico de linha conectando os picos
fig = go.Figure()

# Adicionar a linha conectando os picos
fig.add_trace(go.Scatter(
    x=picos_por_data['dataLiquidacao'],
    y=picos_por_data['taxaCorte'],
    mode='lines+markers',
    name='Picos por Data',
    line=dict(color='blue'),
    marker=dict(size=8, color='blue', symbol='circle')
))

# Personalizar o layout
fig.update_layout(
    title='Gráfico - Taxa de Corte ',
    xaxis_title='Data de Liquidação',
    yaxis_title='Taxa de Corte (%)',
    template='plotly_white',
    xaxis=dict(tickformat='%d/%m/%Y'),
    yaxis=dict(ticksuffix='%'),
    title_font_size=18,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Exibir o gráfico
fig.show()

### Gráfico - Qntd Ofertada x Qntd Aceita

In [7]:
import plotly.graph_objects as go

# Garantir que as colunas 'quantidadeOfertada' e 'quantidadeAceita' sejam convertidas corretamente para numérico
if tabela['quantidadeOfertada'].dtype == 'object':  # Verifica se a coluna é do tipo string
    tabela['quantidadeOfertada'] = tabela['quantidadeOfertada'].str.replace('.', '').astype(float)
else:
    tabela['quantidadeOfertada'] = tabela['quantidadeOfertada'].astype(float)

if tabela['quantidadeAceita'].dtype == 'object':  # Verifica se a coluna é do tipo string
    tabela['quantidadeAceita'] = tabela['quantidadeAceita'].str.replace('.', '').astype(float)
else:
    tabela['quantidadeAceita'] = tabela['quantidadeAceita'].astype(float)

# Garantir que a coluna de datas ('dataLiquidacao') esteja no formato datetime
tabela['dataLiquidacao'] = pd.to_datetime(tabela['dataLiquidacao'], errors='coerce')

# Agrupar por data de liquidação e somar as quantidades ofertada e aceita
dados_agrupados = tabela.groupby('dataLiquidacao', as_index=False).agg({
    'quantidadeOfertada': 'sum',
    'quantidadeAceita': 'sum'
})

# Criar o gráfico de linha com os valores agregados
fig = go.Figure()

# Adicionar a linha para 'quantidadeOfertada'
fig.add_trace(go.Scatter(
    x=dados_agrupados['dataLiquidacao'],
    y=dados_agrupados['quantidadeOfertada'],
    mode='lines+markers',
    name='Quantidade Ofertada',
    line=dict(color='orange'),
    marker=dict(size=8, color='orange', symbol='circle')
))

# Adicionar a linha para 'quantidadeAceita'
fig.add_trace(go.Scatter(
    x=dados_agrupados['dataLiquidacao'],
    y=dados_agrupados['quantidadeAceita'],
    mode='lines+markers',
    name='Quantidade Aceita',
    line=dict(color='blue'),
    marker=dict(size=8, color='blue', symbol='circle')
))

# Personalizar o layout
fig.update_layout(
    title='Gráfico - Quantidade Ofertada e Aceita (Total por Dia)',
    xaxis_title='Data de Liquidação',
    yaxis_title='Quantidade (Total)',
    template='plotly_white',
    xaxis=dict(tickformat='%d/%m/%Y'),
    title_font_size=18,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Exibir o gráfico
fig.show()

### Gráfico - Quantidade Aceita (Venda)

In [25]:
import plotly.graph_objects as go

# Garantir que a coluna 'quantidadeAceita' seja convertida corretamente para numérico
if tabela['quantidadeAceita'].dtype == 'object':  # Verifica se a coluna é do tipo string
    tabela['quantidadeAceita'] = tabela['quantidadeAceita'].str.replace('.', '').astype(float)
else:
    tabela['quantidadeAceita'] = tabela['quantidadeAceita'].astype(float)

# Garantir que a coluna de datas ('dataLiquidacao') esteja no formato datetime
tabela['dataLiquidacao'] = pd.to_datetime(tabela['dataLiquidacao'], errors='coerce')

# Agrupar por data de liquidação e somar a quantidade aceita
quantidade_por_dia = tabela.groupby('dataLiquidacao', as_index=False)['quantidadeAceita'].sum()

# Criar o gráfico de linha com os valores agregados
fig = go.Figure()

# Adicionar a linha conectando os valores de 'quantidadeAceita' por dia
fig.add_trace(go.Scatter(
    x=quantidade_por_dia['dataLiquidacao'],
    y=quantidade_por_dia['quantidadeAceita'],
    mode='lines+markers',
    name='Quantidade Aceita (Total por Dia)',
    line=dict(color='blue'),
    marker=dict(size=8, color='blue', symbol='circle')
))

# Personalizar o layout
fig.update_layout(
    title='Gráfico - Quantidade Aceita (Venda)',
    xaxis_title='Data de Liquidação',
    yaxis_title='Quantidade Aceita (Total)',
    template='plotly_white',
    xaxis=dict(tickformat='%d/%m/%Y'),
    title_font_size=18,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Exibir o gráfico
fig.show()

### Gráfico - Quantidade Ofertada 

In [26]:
import plotly.graph_objects as go

# Garantir que a coluna 'quantidadeOfertada' seja convertida corretamente para numérico
if tabela['quantidadeOfertada'].dtype == 'object':  # Verifica se a coluna é do tipo string
    tabela['quantidadeOfertada'] = tabela['quantidadeOfertada'].str.replace('.', '').astype(float)
else:
    tabela['quantidadeOfertada'] = tabela['quantidadeOfertada'].astype(float)

# Garantir que a coluna de datas ('dataLiquidacao') esteja no formato datetime
tabela['dataLiquidacao'] = pd.to_datetime(tabela['dataLiquidacao'], errors='coerce')

# Agrupar por data de liquidação e somar a quantidade ofertada
ofertada_por_dia = tabela.groupby('dataLiquidacao', as_index=False)['quantidadeOfertada'].sum()

# Criar o gráfico de linha com os valores agregados
fig = go.Figure()

# Adicionar a linha conectando os valores de 'quantidadeOfertada' por dia
fig.add_trace(go.Scatter(
    x=ofertada_por_dia['dataLiquidacao'],
    y=ofertada_por_dia['quantidadeOfertada'],
    mode='lines+markers',
    name='Quantidade Ofertada (Total por Dia)',
    line=dict(color='orange'),
    marker=dict(size=8, color='orange', symbol='circle')
))

# Personalizar o layout
fig.update_layout(
    title='Gráfico - Quantidade Ofertada (Total por Dia)',
    xaxis_title='Data de Liquidação',
    yaxis_title='Quantidade Ofertada (Total)',
    template='plotly_white',
    xaxis=dict(tickformat='%d/%m/%Y'),
    title_font_size=18,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Exibir o gráfico
fig.show()

### Gráfico Financeiro

In [35]:
# Importar bibliotecas necessárias
import pandas as pd
import plotly.graph_objects as go
import ssl

# Configurar o contexto SSL para evitar erros de conexão
ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML do Banco Central
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Ler a tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # Selecionar a primeira tabela da página

# Garantir que a coluna 'financeiro' exista e seja convertida corretamente para numérico
if 'financeiro' in tabela.columns:
    if tabela['financeiro'].dtype == 'object':  # Verifica se a coluna é do tipo string
        tabela['financeiro'] = tabela['financeiro'].str.replace('.', '').astype(float)
    else:
        tabela['financeiro'] = tabela['financeiro'].astype(float)
else:
    raise KeyError("A coluna 'financeiro' não foi encontrada na tabela extraída.")

# Garantir que a coluna de datas ('dataLiquidacao') esteja no formato datetime
tabela['dataLiquidacao'] = pd.to_datetime(tabela['dataLiquidacao'], errors='coerce')

# Agrupar por data de liquidação e somar a coluna 'financeiro'
financeiro_por_dia = tabela.groupby('dataLiquidacao', as_index=False)['financeiro'].sum()

# Criar o gráfico de linha com os valores agregados
fig = go.Figure()

# Adicionar a linha para 'financeiro'
fig.add_trace(go.Scatter(
    x=financeiro_por_dia['dataLiquidacao'],
    y=financeiro_por_dia['financeiro'],
    mode='lines+markers',
    name='Financeiro',
    line=dict(color='green'),
    marker=dict(size=8, color='green', symbol='circle')
))

# Personalizar o layout
fig.update_layout(
    title='Gráfico - Financeiro (em Milhões)',
    xaxis_title='Data de Liquidação',
    yaxis_title='Financeiro (Total)',
    template='plotly_white',
    xaxis=dict(tickformat='%d/%m/%Y'),
    title_font_size=18,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Exibir o gráfico
fig.show()